In [1]:
from __future__ import annotations
from sqlalchemy import create_engine, ForeignKey, UniqueConstraint, select
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped, mapped_column, relationship, Session, sessionmaker

In [2]:
engine = create_engine(url="sqlite:///:memory:")

In [3]:
class Base(DeclarativeBase):
    pass

In [4]:
class User(Base):
    """Represent user"""

    __tablename__ = "user"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str]


class Vote(Base):
    """Represent vote."""

    __tablename__ = "vote"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id = mapped_column(ForeignKey("user.id"))
    target: Mapped[str]
    target_id: Mapped[str]

    __table_args__ = (
        UniqueConstraint("user_id", "target", "target_id"),
    )
    __mapper_args__ = {
        "polymorphic_on": "target",
        "polymorphic_identity": "vote"
    }


class QuestionVote(Vote):
    """Represent vote to question"""

    __mapper_args__ = {
        "polymorphic_identity": "question"
   }

    # 😓 👇👇👇 below.. I don't know how to correctly set up relationship configuration
    question: Mapped[Question] = relationship(
        "Question",
        #primaryjoin="foreign(Question.id) == QuestionVote.target_id"         # 1️⃣ not working ❌(empty)
        #primaryjoin="foreign(Question.id) == remote(QuestionVote.target_id)"  # 2️⃣ not working ❌(empty)
        primaryjoin="Question.id == foreign(QuestionVote.target_id)"         # 3️⃣ not working ❌(empty)
        #primaryjoin="remote(Question.id) == foreign(QuestionVote.target_id)"  # 4️⃣ not working ❌(empty)
     )

class CommentVote(Vote):
    """Represent vote to comment"""

    __mapper_args__ = {
        "polymorphic_identity": "comment"
    }


class Question(Base):
    """Represent question."""

    __tablename__ = "question"

    id: Mapped[int] = mapped_column(primary_key=True)
    user_id = mapped_column(ForeignKey("user.id"))

    votes: Mapped[list[QuestionVote]] = relationship(
        "QuestionVote",
        primaryjoin="foreign(QuestionVote.target_id) == Question.id",
    )

In [5]:
Base.metadata.create_all(bind=engine)

In [6]:
LocalSession = sessionmaker(bind=engine)

In [7]:
with LocalSession() as session:
    user1 = User(name="rodi")
    user2 = User(name="rudy")
    session.add_all([user1, user2])
    session.commit()

/tmp/ipykernel_3625/577691609.py:2: SAWarning: relationship 'Question.votes' will copy column question.id to column vote.target_id, which conflicts with relationship(s): 'QuestionVote.question' (copies question.id to vote.target_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="question"' to the 'Question.votes' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  user1 = User(name="rodi")


In [8]:
with LocalSession() as session:
    rodi = session.get(User, 1)
    q = Question(user_id=rodi.id)
    session.add(q)
    session.commit()

    v = QuestionVote(user_id=rodi.id, target_id=q.id)
    session.add(v)
    session.commit()


In [9]:
with LocalSession() as session:
    q = session.get(Question, 1)
    v = session.get(QuestionVote, 1)
    print(v.question.id)

1
